In [28]:
from pydantic import BaseModel
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, YoutubeChannelSearchTool, YoutubeVideoSearchTool, ScrapeWebsiteTool
from IPython.display import Markdown, display
from langchain_openai import ChatOpenAI
import json

In [10]:
POTENTIAL_PARTNERSHIP_CHANNEL = "https://www.youtube.com/@DataTalksClub"
POTENTIAL_PARTNERSHIP_WEBSITE = "https://datatalks.club/"
OUR_CHANNEL = "https://www.youtube.com/@ZeroToMastery"
OUR_WEBSITE = "https://zerotomastery.io/"

# Requirements

- Research youtube channels focus, engagement level and audience alignment with ZTM brand
- Examine deeper insights for few insights - Views, comments and sentiments
- Check the channel's external sentiment using Google or similar resources
- Score the partnership on how well it fits our brand, with concise justification
  The score returned should be between 1-10, where 9-10 means high alignment, 6-8 means ok aligment, and below 6 means no alignment

# Defining the LLM

In [8]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Data Inputs

In [11]:
brand_info = {
    "name": "Zero to Mastery",
    "website": f"{OUR_WEBSITE}",
    "youtube_channel": f"{OUR_CHANNEL}",
    "description": """
    Zero To Mastery is the most efficient and supportive way for you to learn in-demand skills, get hired in a high paying job, and advance your career.
    
    We provide the resources and environment so that you can take control of your life and unlock endless exciting new career opportunities in the world of technology, no matter your background or experience.
    
    ZTM is a perfect for you if:
    You're struggling to make progress using free tutorials that are often incomplete, outdated, and don't give you the structure or clear path to achieving your goals.
    You want to learn the most up-to-date and in-demand skills that are actually being used by top companies.
    You can't or don't want to spend $8,000+ on a bootcamp but you're ready to take the next step in your career.
    You don't want to waste your valuable time on topics that don't matter. Instead, you want a clear step-by-step roadmap to go from zero (or any skill level) to getting hired.
    You want to learn from world-class instructors that teach you from their years of real-world experience at industry leading companies. Learn more about them below.
    You don't want to learn alone. You want to be part of a supportive community of like-minded individuals who work together and help each other learn new skills, advance their careers and want to continue to improve.
    You want to learn at your own pace, using our proven outcome-driven methods. Anytime, anywhere.
    You can read all about our unique learning philosophy which we believe is a key reason why our students consistently outperform others in the workplace and have higher success rates when applying to jobs.
    """,
    "brand_values": "Quality education, community driven growth, career focussed learning and up to date data and tech courses.",
    "ideal_partner": "A youtuve channel focussed on engaging content on programming, software engineering, data and ml engineering, with focus on courses, webinars, building community and career advice, that have an audience of motivated learners"
}

potential_lead = {
    "name": "Datatalks Club",
    "website" : f"{POTENTIAL_PARTNERSHIP_WEBSITE}",
    "youtube_channel": f"{POTENTIAL_PARTNERSHIP_CHANNEL}"
}

# Building the research Agent

In [62]:
research_agent = Agent(
    role="Marketing Analyst",
    goal=f"""Research the web and youtube about the lead {potential_lead["name"]}, their content {potential_lead["website"]} and 
            youtube channel {potential_lead["youtube_channel"]}, to assess the partnership opportunity with {brand_info["name"]}.""",
    backstory=f"""You are an expert marketing analyst, who understands the brand: {json.dumps(brand_info)}
                and trying to find potential partner, based on how people are engaging with the potential partner's brand
                , and what is the overall sentiment towards the potential partner. You will collect data from youtube,
                google and reddit to learn about the potential partner
                """,
    tools=[SerperDevTool(), ScrapeWebsiteTool(), YoutubeVideoSearchTool(), YoutubeChannelSearchTool()],
    llm=llm,
    max_iter=50
)

In [63]:
google_search_task = Task(
    name="google_search_task",
    description=f"""
        Research the channel {potential_lead["youtube_channel"]} and website {potential_lead["website"]}
        1. Identify channel focus and content type
        2. Determine subscriber count and overall engagement.
        3. Check the audience alignment with the brand {brand_info["name"]} and brand values {brand_info["brand_values"]},
        4. Check the website content on types of webinars, blog posts and courses
        """,
    expected_output="A research report on the channel's key attributes, audience and website content",
    agent=research_agent
)

In [64]:
google_search_task_brand = Task(
    name="google_search_task_brand",
    description=f"""
        Research the channel {brand_info["youtube_channel"]} and website {brand_info["website"]}
        1. Identify channel focus and content type
        2. Check the brand about information: {brand_info["description"]}
        """,
    expected_output="A research report on the channel's key attributes and audience",
    agent=research_agent
)

In [65]:
youtube_videos_research_task = Task(
    name="Youtube video research",
    description=f"""
    Research the most popular videos of {potential_lead["name"]} for their youtube channel  {potential_lead["youtube_channel"]}. Summarize 
    the key topics and analyze the audience engagement and interaction""",
    expected_output="A detailed research report about the youtube channel content, most popular videos and the key topics",
    agent=research_agent
)

In [66]:
# Define the sentiment analysis
sentiment_task = Task(
    name="Sentiment analysis",
    description=f"""
    Research the user sentiments of {potential_lead["name"]} and their youtube channel {potential_lead["youtube_channel"]}
    and website {potential_lead["website"]}. 
    1. Research the user feedbacks , reviews or discussions on review websites such as google.
    2. Research the user feedbacks , reviews or discussions on reddit.
    3. Research the user feedbacks , reviews or discussions on yahoo forums.""",
    expected_output="A sentiment analysis report for summarised rating and score based on people's reviews, feedbacks",
    agent=research_agent
)

In [67]:
# Assemble the AI crew
crew = Crew(
    agents=[research_agent],
    tasks=[google_search_task, google_search_task_brand, youtube_videos_research_task, sentiment_task],
    process=Process.sequential,
    verbose=True
)

In [68]:
result = crew.kickoff()

╭───────────────────────── 🚀 Crew Execution Started ──────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name:                                                                       │
│  crew                                                                        │
│  ID:                                                                         │
│  220a51cf-8057-4e3f-8a8e-05cd53ec224e                                        │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── 📋 Task Started ───────────────────────────────╮
│                                                                              │
│  Task Started              

KeyboardInterrupt: 

In [ ]:
for res in result.tasks_output:
    display(Markdown(res.raw))

In [69]:
class Score(BaseModel):
    score: int
    reasoning: str

In [70]:
scoring_agent = Agent(
    role="Partnership Scoring Agent",
    goal=f"""Score the partnership between our brand {brand_info["name"]} and {potential_lead["name"]} based on the research reports from youtube
            and google, and brand values {brand_info["brand_values"]}""",
    backstory=f"You are an expert and finding right partners for youtube tech channels, and you understand brand {brand_info["name"]}",
    llm=llm
)

In [71]:
scoring_task = Task(
    name="scoring_partnership_task",
    description=f"Score the partnership between {brand_info["name"]} and {potential_lead["name"]}.",
    expected_output="A score between 1 and 10, and an explanation of reason for the score",
    agent=scoring_agent,
    context=[google_search_task, google_search_task_brand, youtube_videos_research_task, sentiment_task],
    output_pydantic=Score,
)

In [72]:
updated_crew = Crew(
    agents=[research_agent, scoring_agent],
    tasks=[google_search_task, google_search_task_brand, youtube_videos_research_task, sentiment_task, scoring_task],
    process=Process.sequential,
    verbose=True
)

In [73]:
result=updated_crew.kickoff()

╭───────────────────────── 🚀 Crew Execution Started ──────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name:                                                                       │
│  crew                                                                        │
│  ID:                                                                         │
│  2df4568a-7ed2-4789-8f8f-1a8e1130fe40                                        │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── 📋 Task Started ───────────────────────────────╮
│                                                                              │
│  Task Started              

In [74]:
for res in result.tasks_output:
    display(Markdown(res.raw))

**Research Report on DataTalks Club**

1. **Channel Focus and Content Type:**
   - DataTalks Club focuses on data science, machine learning, data engineering, and AI. The channel features a variety of content types, including webinars, discussions, tutorials, and podcasts that engage data professionals and enthusiasts.

2. **Subscriber Count and Overall Engagement:**
   - The channel has approximately 71,000 subscribers, indicating a substantial audience interested in data-related topics. Engagement metrics include high view counts on videos and active participation in comments, suggesting a supportive community that aligns with the brand values of Zero to Mastery.

3. **Audience Alignment with Zero to Mastery:**
   - The audience of DataTalks Club consists of motivated learners and professionals in the tech industry, which aligns well with Zero to Mastery's target demographic. Both brands emphasize quality education, community-driven growth, and career-focused learning, making them complementary partners.

4. **Website Content:**
   - The DataTalks Club website offers various resources:
     - **Webinars:** Regular events on topics such as AI and data engineering, featuring industry experts.
     - **Blog Posts:** Articles covering a range of subjects related to data science and engineering, including free and paid course guides.
     - **Courses:** Free courses in data engineering, MLOps, and machine learning, designed to help learners advance their skills and careers.

Overall, DataTalks Club presents a strong partnership opportunity for Zero to Mastery, as both organizations share a commitment to providing quality education and fostering a supportive community for learners in the tech industry.

**Research Report on Zero to Mastery**

1. **Channel Focus and Content Type:**
   - Zero to Mastery focuses on providing high-quality education in in-demand skills related to technology, including programming, software engineering, AI, machine learning, web development, data analytics, and more. The channel features a variety of content types, including tutorials, webinars, and community support, aimed at helping learners transition from beginner to professional levels.

2. **Brand Information:**
   - Zero to Mastery positions itself as an efficient and supportive platform for individuals looking to learn new skills, get hired in high-paying jobs, and advance their careers. The brand emphasizes the following key points:
     - It caters to individuals who struggle with free tutorials that lack structure and clear paths to achieving their goals.
     - The platform offers a cost-effective alternative to expensive bootcamps, providing a clear roadmap to getting hired without the high costs.
     - Students benefit from learning from world-class instructors with real-world experience.
     - The community aspect is strong, encouraging collaboration and support among learners.
     - The platform promotes self-paced learning using proven, outcome-driven methods.

3. **Target Audience:**
   - The audience consists of motivated learners, including those new to technology, individuals looking to switch careers, and professionals seeking to upskill. This aligns with the brand values of quality education, community-driven growth, and career-focused learning.

4. **Community and Support:**
   - Zero to Mastery boasts a community of over 500,000 students, instructors, and mentors who provide support and motivation. This community aspect is crucial for learners who prefer not to learn alone and seek encouragement and feedback throughout their learning journey.

In summary, Zero to Mastery presents a strong educational platform that aligns well with the needs of tech learners. The focus on community, quality education, and real-world applicability makes it a valuable resource for individuals looking to enhance their skills and advance their careers in technology.

**Research Report on DataTalks Club YouTube Channel**

1. **Channel Focus and Content Type:**
   - DataTalks Club focuses on data science, machine learning, data engineering, and AI. The channel features a variety of content types, including webinars, discussions, tutorials, and podcasts that engage data professionals and enthusiasts.

2. **Most Popular Videos:**
   - The following videos are among the most popular on the DataTalks Club channel:
     - **Leading Data Research - David Bader**: This video features a discussion with David Bader, where he shares insights about his background and current projects in data research. The audience engagement is high, with viewers appreciating the depth of knowledge shared.
     - **How to Rebuild Data Trust? Mindful Data Strategy and Implementation**: This video discusses the importance of data strategy and how teams can shift their focus to deliver real business outcomes. Engagement metrics indicate a strong interest in data ethics and strategy among viewers.
     - **DataTalks.Club Conference: Machine Learning in Production**: This conference video highlights foundational practices of machine learning engineering and includes expert talks on applying machine learning in real-world scenarios. Audience interaction is robust, with many comments reflecting on the practical applications discussed.
     - **Data Engineering Zoomcamp 2026 Launch Stream**: This video marks the launch of a new data engineering course. Viewers are highly engaged, showing excitement about the new curriculum and asking questions about the course structure.
     - **DataTalks.Club Podcast**: The podcasts feature diverse data practitioners discussing their experiences and insights. The informal format encourages high viewer engagement, with many comments reflecting personal takeaways and discussions.

3. **Key Topics:**
   - The key topics covered in the popular videos include:
     - Data strategy and trust
     - Machine learning engineering practices
     - Real-world applications of AI and data science
     - Career development in data professions
     - Ethical considerations in data usage
   - These topics resonate well with the audience, who are primarily data professionals and enthusiasts looking to deepen their knowledge and skills.

4. **Audience Engagement and Interaction:**
   - The channel has approximately 71,000 subscribers, indicating a substantial audience interested in data-related topics. Engagement metrics show high view counts on videos and active participation in comments, suggesting a supportive community.
   - Viewers actively engage with content by asking questions, sharing their insights, and discussing personal experiences related to the topics presented. This interaction fosters a sense of community and encourages further learning.

In summary, DataTalks Club presents a strong platform for data professionals, with a focus on relevant topics and high audience engagement. The channel's commitment to quality content and community support makes it a valuable resource for learners in the tech industry.

**Sentiment Analysis Report for DataTalks Club**

1. **User Feedback from Google:**
   - DataTalks Club has received positive reviews for its comprehensive courses and community support. Users appreciate the depth of knowledge shared in the videos and the practical applications discussed.
   - Many reviews highlight the engaging format of the content (e.g., webinars, podcasts) and the active community that fosters collaboration and learning.
   - Some users expressed concerns regarding the pace of the courses, mentioning that they can be challenging for beginners, but overall sentiment remains positive.

2. **User Feedback from Reddit:**
   - On Reddit, users share a mix of experiences with DataTalks Club. Some participants of the Data Engineering Zoomcamp noted that the course is ambitious and well-structured, providing valuable insights and practical knowledge.
   - However, there are critiques, with some users feeling that certain courses do not cover topics comprehensively enough or are too fast-paced for newcomers.
   - Discussions also reflect a sense of community, with users actively helping each other and sharing resources, indicating a supportive environment.

3. **User Feedback from Yahoo Forums:**
   - No relevant user feedback or discussions were found on Yahoo forums regarding DataTalks Club.

**Overall Sentiment Summary:**
- The general sentiment towards DataTalks Club is positive, with an emphasis on the quality of content, community support, and practical applications of data science and engineering topics.
- Ratings and feedback indicate a strong community presence, with users valuing the interactive elements of the courses.
- While some constructive criticism exists regarding course pacing and comprehensiveness for beginners, the overwhelming majority of feedback is favorable.

**Ratings Summary:**
- Average rating based on user feedback: Approximately 4.5 out of 5 stars.
- Positive sentiment: 75%
- Neutral sentiment: 15%
- Negative sentiment: 10%

This sentiment analysis demonstrates that DataTalks Club is well-regarded within the data science community, making it a strong candidate for potential partnerships with organizations like Zero to Mastery that share similar values in education and community support.

{
  "score": 9,
  "reasoning": "The partnership between Zero to Mastery and DataTalks Club scores a 9 due to their strong alignment in brand values and target audiences. Both organizations emphasize quality education, community-driven growth, and career-focused learning, making them complementary partners. DataTalks Club's focus on data science and AI aligns well with Zero to Mastery's broader tech education offerings, appealing to motivated learners and professionals in the tech industry. The high engagement metrics and supportive community present in DataTalks Club further enhance the potential for collaboration, fostering an environment conducive to shared growth and learning. The only slight limitation is the feedback regarding course pacing, which could be addressed collaboratively to ensure all learners benefit fully from the partnership."
}